# Handling Large Result Sets

We can use the `fetchmany` function to specify how many records we want returned by the query. In this way we can work on small batches of records. Useful when you don't have the memory to load all the results at once.

We can then loop over the resulting results proxy, which will return an empty list when there are no more records to be returned. We need to close the proxy afterwards.

In [1]:
from sqlalchemy import create_engine, Table, MetaData, select
from sqlalchemy import and_, or_, not_, between, desc, func
from sqlalchemy import case, cast, Float

# instantiate the database connection
engine = create_engine('sqlite:///../data/sqlalchemy/census.sqlite')
connection = engine.connect()
print('Tables:', engine.table_names())

# instantiate the table obj
census = Table('census', MetaData(), autoload=True, autoload_with=engine)
print('Census:', census.columns.keys())

print('Setup complete!')

Tables: ['census', 'state_fact']
Census: ['state', 'sex', 'age', 'pop2000', 'pop2008']
Setup complete!


**Count the number of times each state appears in the table**

In [3]:
results_proxy = connection.execute(select([census]))

more_results = True
state_count = {}

# Start a while loop checking for more results
while more_results:
    # Fetch the first 50 results from the ResultProxy
    partial_results = results_proxy.fetchmany(50)

    # if empty list, set more_results to False, terminate loop
    if partial_results == []:
        more_results = False

    # Loop over the fetched records and increment the count for the state
    # if  it exists in the dict, otherwise add it.
    for row in partial_results:
        if row.state in state_count:
            state_count[row.state] += 1
        else:
            state_count[row.state] = 1

# Close the ResultProxy, and thus the connection
results_proxy.close()

# Print the count by state
print(state_count)

{'Illinois': 172, 'New Jersey': 172, 'District of Columbia': 172, 'North Dakota': 172, 'Florida': 172, 'Maryland': 172, 'Idaho': 172, 'Massachusetts': 172, 'Oregon': 172, 'Nevada': 172, 'Michigan': 172, 'Wisconsin': 172, 'Missouri': 172, 'Washington': 172, 'North Carolina': 172, 'Arizona': 172, 'Arkansas': 172, 'Colorado': 172, 'Indiana': 172, 'Pennsylvania': 172, 'Hawaii': 172, 'Kansas': 172, 'Louisiana': 172, 'Alabama': 172, 'Minnesota': 172, 'South Dakota': 172, 'New York': 172, 'California': 172, 'Connecticut': 172, 'Ohio': 172, 'Rhode Island': 172, 'Georgia': 172, 'South Carolina': 172, 'Alaska': 172, 'Delaware': 172, 'Tennessee': 172, 'Vermont': 172, 'Montana': 172, 'Kentucky': 172, 'Utah': 172, 'Nebraska': 172, 'West Virginia': 172, 'Iowa': 172, 'Wyoming': 172, 'Maine': 172, 'New Hampshire': 172, 'Mississippi': 172, 'Oklahoma': 172, 'New Mexico': 172, 'Virginia': 172, 'Texas': 172}
